In [ ]:
from sgfmill import sgf, boards, ascii_boards
from sgfmill.boards import Board
from sgfmill.ascii_boards import render_board
import numpy as np
import copy

In [ ]:
file_path = 'go_data/9d/1493996505019999756.sgf'

with open(file_path, 'rb') as f:
    game = sgf.Sgf_game.from_bytes(f.read())

In [ ]:
# empty_idxs = board_arr == None


In [ ]:
def get_boards(file_path): 
    board = Board(19)
    board_list = [copy.deepcopy(board)]
    
    with open(file_path, 'rb') as f:
        game = sgf.Sgf_game.from_bytes(f.read())
        

    for i, node in enumerate(game.get_main_sequence()):

        move = node.get_move() # (color (row, column))
        if move[0] is None or move[1] is None:
            continue
        color = move[0]
        row = move[1][0]
        column = move[1][1]

        board.play(row, column, color)
        board_list.append(copy.deepcopy(board))

    return board_list

In [ ]:
boards = get_boards(file_path)
print(ascii_boards.render_board(boards[-1]))

In [ ]:
def format_board(board):
    board_arr = np.array(board.board)
    board_arr = board_arr.flatten()

    for pos in range(len(board_arr)):
        if board_arr[pos] is None:
            board_arr[pos] = -1
        elif board_arr[pos] == 'w':
            board_arr[pos] = 0
        elif board_arr[pos] == 'b':
            board_arr[pos] = 1
    return board_arr
format_board(boards[-1])

In [ ]:
board_arr

In [ ]:
print(render_board(board))

In [ ]:
def convert_sgf_to_text(sgf_file):
    with open(sgf_file, 'rb') as f:
        game = sgf.Sgf_game.from_bytes(f.read())

    board = np.zeros([19, 19])
    for i, node in enumerate(game.get_main_sequence()):

        print(f"Move {i + 1}:")
        for move in node.properties['B']:
            x, y = ord(move[0]) - ord('a'), ord(move[1]) - ord('a')
            board[x][y] = 1
            print_board(board)
        for move in node.properties['W']:
            x, y = ord(move[0]) - ord('a'), ord(move[1]) - ord('a')
            board[x][y] = -1
            print_board(board)
            
convert_sgf_to_text('go_data/9d/1493996505019999756.sgf')

In [ ]:
def convert_to_onehot(row, column):
    position = np.zeros(19 * 19)
    # row + column * (row length)
    index = row + column * 19
    
    position[index] = 1
    return position



def get_labels(file_path):

    with open(file_path, 'rb') as f:
        game = sgf.Sgf_game.from_bytes(f.read()) 

    # Create labels for training, label = next move
    # Label format: (row, column, color)
    labels = []

    for i, node in enumerate(game.get_main_sequence()):

        move = node.get_move() # (color (row, column))
        if move[0] is None or move[1] is None:
            continue
        color = move[0]
        row = move[1][0]
        column = move[1][1]
        labels.append(convert_to_onehot(row, column))
    return labels
        
get_labels(file_path)